In [7]:
# dfs concurrency
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
import threading

def dfs(url):
    visited = set([url])
    dq = deque([[url, "", 0]])
    max_depth = 3
    max_breadth = 5
    string = ""
    while dq:
        base, path, depth = dq.popleft()
        if depth < max_depth:
            try:
                soup = BeautifulSoup(requests.get(base + path).text, "html.parser")
                links = soup.find_all("a")[:max_breadth]
                for link in links:
                    href = link.get("href")
                    if href not in visited:
                        visited.add(href)
                        string += ("  " * depth + f"at depth {depth}: {href}\n")

                        if href.startswith("http") or href.startswith("https"):
                            dq.append([href, "", depth + 1])
                        else:
                            dq.append([base, href, depth + 1])
            except:
                pass

    filename =""
    if "https://" in url:
        filename = re.sub(r"https://","",url)
        filename = re.sub(r".com","",filename)
    elif "http://" in url:
        filename = re.sub(r"http://","",url)
        filename = re.sub(r".com","",filename)

    file  = open("crawled_links/"+filename+".txt","w")
    file.write(string)
    file.close()

In [8]:
frontier = ["http://toscrape.com", "https://soundcloud.com", "http://reddit.com", "https://fc2.com"]

def threadCreater(frontier):
    threads = []
    for i in frontier:
        t = threading.Thread(target = dfs, args=(i,))
        threads.append(t)
    return threads

threads = threadCreater(frontier)
for i in threads:
    i.start()
for i in threads:
    i.join()
print("files made!")

files made!


In [20]:
import math

def unary(x):
    return '0'*(x-1)+'1'

def encoder(x, b):
    q = math.floor(x/b)
    q_encode = unary(q+1)
    i = math.floor(math.log2(b))
    d = int(2**(i+1))-b
    r = x%b
    rem = ""
    if r < d:
        rem = bin(r)[2:]
        l = len(rem)
        if l < i:
            rem = '0'*(i - l)+rem
    else:
        rem = bin(r + d)[2:]
        l = len(rem)
        if l < i+1:
            rem = '0'*(i+1-l) + rem
    return q_encode+rem

print(encoder(74, 16))
print(encoder(50, 11))
print(encoder(9,10))

000011010
000011011
11111


In [10]:
def decode(n, M):
    q=len(n.split('1')[0])
    b=math.floor(math.log2(M))
    k = 2**(b+1)-M
    r = int(n[q+1:q+1+b],2)
    if r>=k:
        r = int(n[q+1:q+1+b+1],2)
        r=r-k
    x=q*M+r
    return x

print(decode("1111111110010001101", 10))

9


In [11]:
# Q4
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from functools import reduce
from urllib import request
import pandas as pd

stop_words = set(STOP_WORDS)
stop_words = stop_words.union({'.',',','\'','\"','?','{','}','[',']','<','>','(',')','!'})

def extract(url):
    html = request.urlopen(url).read().decode('utf8')
    string = BeautifulSoup(html, 'html.parser').get_text().lower()
    return list(word_tokenize(string))

def store(name, ls):
    with open(name,'w') as f:
        for item in list(ls):
            try:
                f.write('%s\n'%item)
            except:
                pass
def term_maker(ls, filenames):
    terms = []
    index = 0
    for i in ls:
        st = set(i).difference(stop_words)
        store('extracted_content/'+filenames[index]+'.txt', list(st))
        index += 1
        terms.append(list(st))
    terms = reduce(lambda z,y : z+y, terms)
    return terms

def counter(tokens, terms):
    ls = []
    for i in terms:
        ls.append(tokens.count(i))
    return ls

base = 'https://en.wikipedia.org/wiki/'
links = ['Web_mining','Data_mining','Artificial_intelligence','Machine_learning','Natural_language_processing','Text_mining']
token_list = []
for i in links:
    token_list.append(extract(base+i))
terms = term_maker(token_list, links)
bagOfWords = []
for i in token_list:
    bagOfWords.append(counter(i,terms))

pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : bagOfWords[0],     
    "Doc 2" : bagOfWords[1],     
    "Doc 3" : bagOfWords[2],     
    "Doc 4" : bagOfWords[3],     
    "Doc 5" : bagOfWords[4],     
    "Doc 6" : bagOfWords[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,masand,2,0,0,0,0,0
1,kurt,1,0,3,0,0,0
2,vol.3,1,0,0,0,0,0
3,vazirgiannis,1,0,0,0,0,0
4,cs.lg/0011033,1,0,0,0,0,0
...,...,...,...,...,...,...,...
15125,roll,0,0,1,0,0,1
15126,social,0,4,21,5,1,10
15127,nature,1,0,8,7,1,3
15128,linking,0,0,0,0,0,2


In [12]:
def total(ls):
    count=0
    for i in ls:
        if i != 0:
            count+=1
    return count

TF = [[number/total(i) for number in i] for i in bagOfWords]
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF[0],     
    "Doc 2" : TF[1],     
    "Doc 3" : TF[2],     
    "Doc 4" : TF[3],     
    "Doc 5" : TF[4],     
    "Doc 6" : TF[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,masand,0.000576,0.000000,0.000000,0.000000,0.000000,0.000000
1,kurt,0.000288,0.000000,0.000273,0.000000,0.000000,0.000000
2,vol.3,0.000288,0.000000,0.000000,0.000000,0.000000,0.000000
3,vazirgiannis,0.000288,0.000000,0.000000,0.000000,0.000000,0.000000
4,cs.lg/0011033,0.000288,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
15125,roll,0.000000,0.000000,0.000091,0.000000,0.000000,0.000194
15126,social,0.000000,0.000706,0.001913,0.000645,0.000186,0.001937
15127,nature,0.000288,0.000000,0.000729,0.000903,0.000186,0.000581
15128,linking,0.000000,0.000000,0.000000,0.000000,0.000000,0.000387


In [13]:
import math
def IDF_helper(ls):
    total = []
    length = len(ls[0])
    for i in range(length):
        count = 0
        for j in range(len(ls)):
            value = ls[j][i]
            if value != 0:
                count += 1
        total.append(count)
    return total

def IDF_calc():
    helper = IDF_helper(bagOfWords)
    no_of_doc = 6
    IDF_ls = []
    for i in helper:
        IDF_ls.append(math.log((1+no_of_doc)/i))
    return IDF_ls
pd.DataFrame({
    "Terms" : list(terms),
    "IDF value" : IDF_calc()
})

,Terms,IDF value
0,masand,1.945910
1,kurt,1.252763
2,vol.3,1.945910
3,vazirgiannis,1.945910
4,cs.lg/0011033,1.945910
...,...,...
15125,roll,1.252763
15126,social,0.336472
15127,nature,0.336472
15128,linking,1.945910


In [14]:
def TF_IDF_calc(ls):
    IDF = IDF_calc()
    for i in range(len(ls[0])):
        for j in range(6):
                ls[j][i]*=IDF[i]
    return ls
TF_IDF = TF_IDF_calc(TF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF_IDF[0],
    "Doc 2" : TF_IDF[1],
    "Doc 3" : TF_IDF[2],
    "Doc 4" : TF_IDF[3],
    "Doc 5" : TF_IDF[4],
    "Doc 6" : TF_IDF[5]
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,masand,0.001121,0.000000,0.000000,0.000000,0.000000,0.000000
1,kurt,0.000361,0.000000,0.000342,0.000000,0.000000,0.000000
2,vol.3,0.000560,0.000000,0.000000,0.000000,0.000000,0.000000
3,vazirgiannis,0.000560,0.000000,0.000000,0.000000,0.000000,0.000000
4,cs.lg/0011033,0.000560,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
15125,roll,0.000000,0.000000,0.000114,0.000000,0.000000,0.000243
15126,social,0.000000,0.000238,0.000644,0.000217,0.000063,0.000652
15127,nature,0.000097,0.000000,0.000245,0.000304,0.000063,0.000196
15128,linking,0.000000,0.000000,0.000000,0.000000,0.000000,0.000754


In [15]:
def query_TF_IDF(string):
    ref_tokens = list(word_tokenize(string.lower()))
    tokens = list(set(word_tokenize(string.lower())).difference(stop_words))
    Tf_query = []
    for i in tokens:
        Tf_query.append(1/ref_tokens.count(i))
    ref_IDF = IDF_calc()
    TF_IDF = []
    for i in range(len(tokens)):
        if tokens[i] in terms:
            index = terms.index(tokens[i])
            value = ref_IDF[index]*Tf_query[i]
            TF_IDF.append(value)
        else:
            TF_IDF.append(0)
    return (tokens,TF_IDF)

string = "I want communications based on machine learning"
Q_TF_IDF = query_TF_IDF(string)
pd.DataFrame({
    "Terms" : Q_TF_IDF[0],
    "TF_IDF" : Q_TF_IDF[1]
})

,Terms,TF_IDF
0,learning,0.154151
1,machine,0.154151
2,communications,1.252763
3,want,1.945910
4,based,0.154151


In [16]:
def normalization_Document(TF_IDF):
    no_of_docs = len(TF_IDF)
    for i in range(no_of_docs):
        divider = 0
        for j in TF_IDF[i]:
            divider += j**2
        divider = math.sqrt(divider)
        TF_IDF[i] = [k/divider for k in TF_IDF[i]]
    return TF_IDF

normalized = normalization_Document(TF_IDF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : normalized[0],
    "Doc 2" : normalized[1],
    "Doc 3" : normalized[2],
    "Doc 4" : normalized[3],
    "Doc 5" : normalized[4],
    "Doc 6" : normalized[5],
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,masand,0.035399,0.000000,0.000000,0.000000,0.00000,0.000000
1,kurt,0.011395,0.000000,0.004518,0.000000,0.00000,0.000000
2,vol.3,0.017700,0.000000,0.000000,0.000000,0.00000,0.000000
3,vazirgiannis,0.017700,0.000000,0.000000,0.000000,0.00000,0.000000
4,cs.lg/0011033,0.017700,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...
15125,roll,0.000000,0.000000,0.001506,0.000000,0.00000,0.008604
15126,social,0.000000,0.007513,0.008494,0.006060,0.00189,0.023110
15127,nature,0.003060,0.000000,0.003236,0.008483,0.00189,0.006933
15128,linking,0.000000,0.000000,0.000000,0.000000,0.00000,0.026730


In [17]:
def normalization_Query(string):
    TF_IDF = query_TF_IDF(string)[1]
    divider = 0
    for i in TF_IDF:
        divider += i**2
    TF_IDF = [i/divider for i in TF_IDF]
    return TF_IDF

pd.DataFrame({
    "Terms" : query_TF_IDF(string)[0],
    "TF_IDF" : normalization_Query(string)
})

,Terms,TF_IDF
0,learning,0.028403
1,machine,0.028403
2,communications,0.230828
3,want,0.358543
4,based,0.028403


In [19]:
def cosine_dist():
    TF_IDF_doc = normalization_Document(TF_IDF)
    TF_IDF_query = normalization_Query(string)
    tokens = Q_TF_IDF[0]
    index_ls = []
    for i in tokens:
        index_ls.append(terms.index(i))

    cosine_rec = []
    for i in range(len(TF_IDF_doc)):
        cosine_dist = 0
        index = 0
        for j in index_ls:
            cosine_dist += TF_IDF_doc[i][j]* TF_IDF_query[index]
        cosine_rec.append(cosine_dist)
    return cosine_rec

print(cosine_dist())

[0.0011650218336673745, 0.0022486810402766876, 0.0012770095255518572, 0.007916587483100508, 0.0016481390137395892, 0.000661575832315287]
